# Iteration 2: 

1. Automatically download the US Stock market indexs: Down Jones, NASDAQ, RUSSEL 2000 and S&P 500 from Yahoo Finance into CSV files


# 1)ScrapingYahoo Finance data

Reference:

https://stackoverflow.com/questions/44225771/scraping-historical-data-from-yahoo-finance-with-python

In [12]:
import re
from io import StringIO
from datetime import datetime, timedelta

import requests
import pandas as pd

#https://finance.yahoo.com/quote/%5EDJI/history?p=%5EDJI

class YahooFinanceHistory:
    timeout = 2
    crumb_link = 'https://finance.yahoo.com/quote/{0}/history?p={0}'
    crumble_regex = r'CrumbStore":{"crumb":"(.*?)"}'
    quote_link = 'https://query1.finance.yahoo.com/v7/finance/download/{quote}?period1={dfrom}&period2={dto}&interval=1d&events=history&crumb={crumb}'

    def __init__(self, symbol, days_back=7):
        self.symbol = symbol
        self.session = requests.Session()
        self.dt = timedelta(days=days_back)

    def get_crumb(self):
        response = self.session.get(self.crumb_link.format(self.symbol), timeout=self.timeout)
        response.raise_for_status()
        match = re.search(self.crumble_regex, response.text)
        if not match:
            raise ValueError('Could not get crumb from Yahoo Finance')
        else:
            self.crumb = match.group(1)

    def get_quote(self):
        if not hasattr(self, 'crumb') or len(self.session.cookies) == 0:
            self.get_crumb()
        now = datetime.utcnow()
        dateto = int(now.timestamp())
        datefrom = int((now - self.dt).timestamp())
        url = self.quote_link.format(quote=self.symbol, dfrom=datefrom, dto=dateto, crumb=self.crumb)
        response = self.session.get(url)
        response.raise_for_status()
        return pd.read_csv(StringIO(response.text), parse_dates=['Date'])


In [13]:
NASDAQ = YahooFinanceHistory('NDAQ', days_back=3560).get_quote()
SNP = YahooFinanceHistory('^GSPC', days_back=3560).get_quote()
RUSSEL= YahooFinanceHistory('^RUT', days_back=3560).get_quote()
DOWJONES = YahooFinanceHistory('^DJI', days_back=3560).get_quote()


In [14]:
NASDAQ.head()

,Date,Open,High,Low,Close,Adj Close,Volume
0,2009-02-25,20.990000,21.650000,20.080000,20.959999,18.666372,3930900
1,2009-02-26,22.510000,23.250000,21.020000,21.860001,19.467888,6929900
2,2009-02-27,21.290001,21.820000,20.879999,20.900000,18.612944,3623000
3,2009-03-02,20.280001,21.200001,19.690001,19.860001,17.686747,3860300
4,2009-03-03,20.240000,20.500000,18.490000,18.670000,16.626970,6663900


# set index field at dataframe

In [15]:
NASDAQ.set_index('Date', inplace=True)

### MISTAKE: ALWAYS BECAUSE OF CASESENSIVE 

In [16]:
SNP.set_index('Date', inplace=True)
RUSSEL.set_index('Date', inplace=True)
DOWJONES.set_index('Date', inplace=True)

In [17]:
NASDAQ.index

DatetimeIndex(['2009-02-25', '2009-02-26', '2009-02-27', '2009-03-02',
               '2009-03-03', '2009-03-04', '2009-03-05', '2009-03-06',
               '2009-03-09', '2009-03-10',
               ...
               '2018-11-09', '2018-11-12', '2018-11-13', '2018-11-14',
               '2018-11-15', '2018-11-16', '2018-11-19', '2018-11-20',
               '2018-11-21', '2018-11-23'],
              dtype='datetime64[ns]', name='Date', length=2456, freq=None)

# rename the column at dataframe

In [18]:
SNP=SNP.rename(columns={'Adj Close':'Adj_Close'})


In [19]:
SNP.head()

,Open,High,Low,Close,Adj_Close,Volume
Date,,,,,,
2009-02-25,770.640015,780.119995,752.890015,764.900024,764.900024,7483640000
2009-02-26,765.760010,779.419983,751.750000,752.830017,752.830017,7599970000
2009-02-27,749.929993,751.270020,734.520020,735.090027,735.090027,8926480000
2009-03-02,729.570007,729.570007,699.700012,700.820007,700.820007,7868290000
2009-03-03,704.440002,711.669983,692.299988,696.330017,696.330017,7583230000


In [20]:

SNP['Day_MS_1_Open'] = SNP['Open'].shift(1)
SNP['Day_MS_1_High'] = SNP['High'].shift(1)
SNP['Day_MS_1_Low'] = SNP['Low'].shift(1)
SNP['Day_MS_1_Close'] = SNP['Close'].shift(1)
SNP['Day_MS_1_Adj_Close'] = SNP['Adj_Close'].shift(1)
SNP['Day_MS_1_Volume'] = SNP['Volume'].shift(1)
SNP['Day_MS_2_Open'] = SNP['Open'].shift(2)
SNP['Day_MS_2_High'] = SNP['High'].shift(2)
SNP['Day_MS_2_Low'] = SNP['Low'].shift(2)
SNP['Day_MS_2_Close'] = SNP['Close'].shift(2)
SNP['Day_MS_2_Adj_Close'] = SNP['Adj_Close'].shift(2)
SNP['Day_MS_2_Volume'] = SNP['Volume'].shift(2)
SNP['Day_MS_3_Open'] = SNP['Open'].shift(3)
SNP['Day_MS_3_High'] = SNP['High'].shift(3)
SNP['Day_MS_3_Low'] = SNP['Low'].shift(3)
SNP['Day_MS_3_Close'] = SNP['Close'].shift(3)
SNP['Day_MS_3_Adj_Close'] = SNP['Adj_Close'].shift(3)
SNP['Day_MS_3_Volume'] = SNP['Volume'].shift(3)
SNP['Day_MS_4_Open'] = SNP['Open'].shift(4)
SNP['Day_MS_4_High'] = SNP['High'].shift(4)
SNP['Day_MS_4_Low'] = SNP['Low'].shift(4)
SNP['Day_MS_4_Close'] = SNP['Close'].shift(4)
SNP['Day_MS_4_Adj_Close'] = SNP['Adj_Close'].shift(4)
SNP['Day_MS_4_Volume'] = SNP['Volume'].shift(4)
SNP['Day_MS_5_Open'] = SNP['Open'].shift(5)
SNP['Day_MS_5_High'] = SNP['High'].shift(5)
SNP['Day_MS_5_Low'] = SNP['Low'].shift(5)
SNP['Day_MS_5_Close'] = SNP['Close'].shift(5)
SNP['Day_MS_5_Adj_Close'] = SNP['Adj_Close'].shift(5)
SNP['Day_MS_5_Volume'] = SNP['Volume'].shift(5)


In [21]:
SNP.head()

,Open,High,Low,Close,Adj_Close,Volume,Day_MS_1_Open,Day_MS_1_High,Day_MS_1_Low,Day_MS_1_Close,...,Day_MS_4_Low,Day_MS_4_Close,Day_MS_4_Adj_Close,Day_MS_4_Volume,Day_MS_5_Open,Day_MS_5_High,Day_MS_5_Low,Day_MS_5_Close,Day_MS_5_Adj_Close,Day_MS_5_Volume
Date,,,,,,,,,,,,,,,,,,,,,
2009-02-25,770.640015,780.119995,752.890015,764.900024,764.900024,7483640000,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2009-02-26,765.760010,779.419983,751.750000,752.830017,752.830017,7599970000,770.640015,780.119995,752.890015,764.900024,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2009-02-27,749.929993,751.270020,734.520020,735.090027,735.090027,8926480000,765.760010,779.419983,751.750000,752.830017,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2009-03-02,729.570007,729.570007,699.700012,700.820007,700.820007,7868290000,749.929993,751.270020,734.520020,735.090027,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2009-03-03,704.440002,711.669983,692.299988,696.330017,696.330017,7583230000,729.570007,729.570007,699.700012,700.820007,...,752.890015,764.900024,764.900024,7.483640e+09,NaN,NaN,NaN,NaN,NaN,NaN


In [22]:
SNP.to_csv(r"C:\Users\rz08\Desktop\Sammie\Learn\Practise\Stocks\Data\Iterations\Iteration2\SNP.csv")

In [23]:
NASDAQ=NASDAQ.rename(columns={'Adj Close':'Adj_Close'})
NASDAQ['Day_MS_1_Open'] = NASDAQ['Open'].shift(1)
NASDAQ['Day_MS_1_High'] = NASDAQ['High'].shift(1)
NASDAQ['Day_MS_1_Low'] = NASDAQ['Low'].shift(1)
NASDAQ['Day_MS_1_Close'] = NASDAQ['Close'].shift(1)
NASDAQ['Day_MS_1_Adj_Close'] = NASDAQ['Adj_Close'].shift(1)
NASDAQ['Day_MS_1_Volume'] = NASDAQ['Volume'].shift(1)
NASDAQ['Day_MS_2_Open'] = NASDAQ['Open'].shift(2)
NASDAQ['Day_MS_2_High'] = NASDAQ['High'].shift(2)
NASDAQ['Day_MS_2_Low'] = NASDAQ['Low'].shift(2)
NASDAQ['Day_MS_2_Close'] = NASDAQ['Close'].shift(2)
NASDAQ['Day_MS_2_Adj_Close'] = NASDAQ['Adj_Close'].shift(2)
NASDAQ['Day_MS_2_Volume'] = NASDAQ['Volume'].shift(2)
NASDAQ['Day_MS_3_Open'] = NASDAQ['Open'].shift(3)
NASDAQ['Day_MS_3_High'] = NASDAQ['High'].shift(3)
NASDAQ['Day_MS_3_Low'] = NASDAQ['Low'].shift(3)
NASDAQ['Day_MS_3_Close'] = NASDAQ['Close'].shift(3)
NASDAQ['Day_MS_3_Adj_Close'] = NASDAQ['Adj_Close'].shift(3)
NASDAQ['Day_MS_3_Volume'] = NASDAQ['Volume'].shift(3)
NASDAQ['Day_MS_4_Open'] = NASDAQ['Open'].shift(4)
NASDAQ['Day_MS_4_High'] = NASDAQ['High'].shift(4)
NASDAQ['Day_MS_4_Low'] = NASDAQ['Low'].shift(4)
NASDAQ['Day_MS_4_Close'] = NASDAQ['Close'].shift(4)
NASDAQ['Day_MS_4_Adj_Close'] = NASDAQ['Adj_Close'].shift(4)
NASDAQ['Day_MS_4_Volume'] = NASDAQ['Volume'].shift(4)
NASDAQ['Day_MS_5_Open'] = NASDAQ['Open'].shift(5)
NASDAQ['Day_MS_5_High'] = NASDAQ['High'].shift(5)
NASDAQ['Day_MS_5_Low'] = NASDAQ['Low'].shift(5)
NASDAQ['Day_MS_5_Close'] = NASDAQ['Close'].shift(5)
NASDAQ['Day_MS_5_Adj_Close'] = NASDAQ['Adj_Close'].shift(5)
NASDAQ['Day_MS_5_Volume'] = NASDAQ['Volume'].shift(5)
NASDAQ.head()

NASDAQ.to_csv(r"C:\Users\rz08\Desktop\Sammie\Learn\Practise\Stocks\Data\Iterations\Iteration2\NASDAQ.csv")


In [24]:
RUSSEL=RUSSEL.rename(columns={'Adj Close':'Adj_Close'})
RUSSEL.head()

RUSSEL['Day_MS_1_Open'] = RUSSEL['Open'].shift(1)
RUSSEL['Day_MS_1_High'] = RUSSEL['High'].shift(1)
RUSSEL['Day_MS_1_Low'] = RUSSEL['Low'].shift(1)
RUSSEL['Day_MS_1_Close'] = RUSSEL['Close'].shift(1)
RUSSEL['Day_MS_1_Adj_Close'] = RUSSEL['Adj_Close'].shift(1)
RUSSEL['Day_MS_1_Volume'] = RUSSEL['Volume'].shift(1)
RUSSEL['Day_MS_2_Open'] = RUSSEL['Open'].shift(2)
RUSSEL['Day_MS_2_High'] = RUSSEL['High'].shift(2)
RUSSEL['Day_MS_2_Low'] = RUSSEL['Low'].shift(2)
RUSSEL['Day_MS_2_Close'] = RUSSEL['Close'].shift(2)
RUSSEL['Day_MS_2_Adj_Close'] = RUSSEL['Adj_Close'].shift(2)
RUSSEL['Day_MS_2_Volume'] = RUSSEL['Volume'].shift(2)
RUSSEL['Day_MS_3_Open'] = RUSSEL['Open'].shift(3)
RUSSEL['Day_MS_3_High'] = RUSSEL['High'].shift(3)
RUSSEL['Day_MS_3_Low'] = RUSSEL['Low'].shift(3)
RUSSEL['Day_MS_3_Close'] = RUSSEL['Close'].shift(3)
RUSSEL['Day_MS_3_Adj_Close'] = RUSSEL['Adj_Close'].shift(3)
RUSSEL['Day_MS_3_Volume'] = RUSSEL['Volume'].shift(3)
RUSSEL['Day_MS_4_Open'] = RUSSEL['Open'].shift(4)
RUSSEL['Day_MS_4_High'] = RUSSEL['High'].shift(4)
RUSSEL['Day_MS_4_Low'] = RUSSEL['Low'].shift(4)
RUSSEL['Day_MS_4_Close'] = RUSSEL['Close'].shift(4)
RUSSEL['Day_MS_4_Adj_Close'] = RUSSEL['Adj_Close'].shift(4)
RUSSEL['Day_MS_4_Volume'] = RUSSEL['Volume'].shift(4)
RUSSEL['Day_MS_5_Open'] = RUSSEL['Open'].shift(5)
RUSSEL['Day_MS_5_High'] = RUSSEL['High'].shift(5)
RUSSEL['Day_MS_5_Low'] = RUSSEL['Low'].shift(5)
RUSSEL['Day_MS_5_Close'] = RUSSEL['Close'].shift(5)
RUSSEL['Day_MS_5_Adj_Close'] = RUSSEL['Adj_Close'].shift(5)
RUSSEL['Day_MS_5_Volume'] = RUSSEL['Volume'].shift(5)

RUSSEL.to_csv(r"C:\Users\rz08\Desktop\Sammie\Learn\Practise\Stocks\Data\Iterations\Iteration2\RUSSEL.csv")


In [25]:
DOWJONES=DOWJONES.rename(columns={'Adj Close':'Adj_Close'})
DOWJONES.head()

DOWJONES['Day_MS_1_Open'] = DOWJONES['Open'].shift(1)
DOWJONES['Day_MS_1_High'] = DOWJONES['High'].shift(1)
DOWJONES['Day_MS_1_Low'] = DOWJONES['Low'].shift(1)
DOWJONES['Day_MS_1_Close'] = DOWJONES['Close'].shift(1)
DOWJONES['Day_MS_1_Adj_Close'] = DOWJONES['Adj_Close'].shift(1)
DOWJONES['Day_MS_1_Volume'] = DOWJONES['Volume'].shift(1)
DOWJONES['Day_MS_2_Open'] = DOWJONES['Open'].shift(2)
DOWJONES['Day_MS_2_High'] = DOWJONES['High'].shift(2)
DOWJONES['Day_MS_2_Low'] = DOWJONES['Low'].shift(2)
DOWJONES['Day_MS_2_Close'] = DOWJONES['Close'].shift(2)
DOWJONES['Day_MS_2_Adj_Close'] = DOWJONES['Adj_Close'].shift(2)
DOWJONES['Day_MS_2_Volume'] = DOWJONES['Volume'].shift(2)
DOWJONES['Day_MS_3_Open'] = DOWJONES['Open'].shift(3)
DOWJONES['Day_MS_3_High'] = DOWJONES['High'].shift(3)
DOWJONES['Day_MS_3_Low'] = DOWJONES['Low'].shift(3)
DOWJONES['Day_MS_3_Close'] = DOWJONES['Close'].shift(3)
DOWJONES['Day_MS_3_Adj_Close'] = DOWJONES['Adj_Close'].shift(3)
DOWJONES['Day_MS_3_Volume'] = DOWJONES['Volume'].shift(3)
DOWJONES['Day_MS_4_Open'] = DOWJONES['Open'].shift(4)
DOWJONES['Day_MS_4_High'] = DOWJONES['High'].shift(4)
DOWJONES['Day_MS_4_Low'] = DOWJONES['Low'].shift(4)
DOWJONES['Day_MS_4_Close'] = DOWJONES['Close'].shift(4)
DOWJONES['Day_MS_4_Adj_Close'] = DOWJONES['Adj_Close'].shift(4)
DOWJONES['Day_MS_4_Volume'] = DOWJONES['Volume'].shift(4)
DOWJONES['Day_MS_5_Open'] = DOWJONES['Open'].shift(5)
DOWJONES['Day_MS_5_High'] = DOWJONES['High'].shift(5)
DOWJONES['Day_MS_5_Low'] = DOWJONES['Low'].shift(5)
DOWJONES['Day_MS_5_Close'] = DOWJONES['Close'].shift(5)
DOWJONES['Day_MS_5_Adj_Close'] = DOWJONES['Adj_Close'].shift(5)
DOWJONES['Day_MS_5_Volume'] = DOWJONES['Volume'].shift(5)

DOWJONES.to_csv(r"C:\Users\rz08\Desktop\Sammie\Learn\Practise\Stocks\Data\Iterations\Iteration2\DOWJONES.csv")
